# Code to download peat rasters from ArcGIS REST Server

By André

First chunk is for running on colab. Second is the same code but made to run locally.

Code is to download Forest Service peat rasters:

[histosol_top20crops](https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/histosol_top20crops/ImageServer)

[histosol_percents](https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/histosol_percents/ImageServer)

# Download data for N Carolina extent (Run on Google COLAB)

In [ ]:
# Ensure necessary libraries are installed and suppress output
!pip install requests > /dev/null
!pip install rasterio > /dev/null
!pip install geopandas > /dev/null
!pip install tqdm > /dev/null

from google.colab import drive

# Mount Google Drive to access your files
drive.mount('/content/drive')

In [ ]:
import requests
import os
import math
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gpd
from time import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm
from google.colab import drive

# This script downloads raster data for a specified extent as smaller GeoTIFF
# tiles, clips them to the AOI, merges them into a single GeoTIFF file, and
# deletes the individual tiles.

# Define the path to your Google Drive directory
output_directory = "/content/drive/My Drive/TerraCarbon/Code_Andre/peat"
tiles_directory = os.path.join(output_directory, "tiles")

# Data name to download
#data_name = "histosol_percents"
data_name = "histosol_top20crops"

# Define the URL for the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{data_name}/ImageServer/exportImage"

# Define AOI name:
aoi_name = 'SE'
#aoi_name = 'USA'

# Define AOI GeoJSON file path
aoi_geojson_path = "C:/TerraCarbon/peat/data/SE_peat_aoi.geojson"
#aoi_geojson_path = "C:/TerraCarbon/peat/data/US_peat_aoi.geojson"

# Read AOI GeoJSON to get the geometry
aoi = gpd.read_file(aoi_geojson_path)

# Ensure the AOI is in the same projection as the data (EPSG:3857)
if aoi.crs.to_string() != 'EPSG:3857':
    aoi = aoi.to_crs('EPSG:3857')

# Make the aoi into a bounding box
aoi_bounds = aoi.total_bounds  # [xmin, ymin, xmax, ymax]

# Convert the bounds to a dictionary
bbox = {
    "xmin": aoi_bounds[0],
    "ymin": aoi_bounds[1],
    "xmax": aoi_bounds[2],
    "ymax": aoi_bounds[3]
}

# Pixel size
pixel_size = 30

# Tile size constraints (in pixels)
tile_width = 15000
tile_height = 4100

# Calculate the width and height in meters
tile_width_m = tile_width * pixel_size
tile_height_m = tile_height * pixel_size

# Calculate the number of tiles needed
width = bbox["xmax"] - bbox["xmin"]
height = bbox["ymax"] - bbox["ymin"]
tiles_x = math.ceil(width / tile_width_m)
tiles_y = math.ceil(height / tile_height_m)


# Ensure the directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(tiles_directory, exist_ok=True)

# List to store file paths of downloaded tiles
tile_paths = []

# Setup retry strategy
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)

# Loop over tiles and download each one
total_tiles = tiles_x * tiles_y
start_time = time()

for i in tqdm(range(tiles_x), desc="Downloading tiles", unit="tile"):
    for j in range(tiles_y):
        xmin = bbox["xmin"] + i * tile_width_m
        xmax = min(xmin + tile_width_m, bbox["xmax"])
        ymin = bbox["ymin"] + j * tile_height_m
        ymax = min(ymin + tile_height_m, bbox["ymax"])

        # Calculate the actual tile size in pixels for the request
        actual_tile_width = int((xmax - xmin) / pixel_size)
        actual_tile_height = int((ymax - ymin) / pixel_size)

        # Define the parameters for the exportImage request
        params = {
            "bbox": f"{xmin},{ymin},{xmax},{ymax}",
            "bboxSR": 102100,
            "size": f"{actual_tile_width},{actual_tile_height}",
            "imageSR": 102100,
            "format": "tiff",
            "f": "image"
        }

        # Send the GET request to the server
        try:
            response = http.get(url, params=params, timeout=120)
            response.raise_for_status()  # Raise an exception for HTTP errors

            # Define the output path for the GeoTIFF tile
            tile_path = os.path.join(tiles_directory, f"tile_{i}_{j}.tiff")
            tile_path = os.path.normpath(tile_path)  # Normalize the path to use the correct OS separator

            # Save the GeoTIFF file to the specified directory
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            tile_paths.append(tile_path)
            print(f"Tile ({i}, {j}) successfully downloaded and saved as '{tile_path}'")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download tile ({i}, {j}). Error: {e}")
            print(f"Response content: {response.content if response else 'No response content'}")

# Clip tiles to AOI and save the clipped tiles
clipped_tile_paths = []

for tile_path in tile_paths:
    with rasterio.open(tile_path) as src:
        try:
            out_image, out_transform = mask(src, aoi.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            clipped_tile_path = tile_path.replace(".tiff", f"_{aoi_name}.tiff")
            with rasterio.open(clipped_tile_path, "w", **out_meta) as dest:
                dest.write(out_image)
            clipped_tile_paths.append(clipped_tile_path)
            print(f"Tile {tile_path} successfully clipped and saved as '{clipped_tile_path}'")
        except Exception as e:
            print(f"Failed to clip tile {tile_path}. Error: {e}")

# Merge clipped tiles into a single image
if clipped_tile_paths:
    src_files_to_mosaic = [rasterio.open(fp) for fp in clipped_tile_paths]
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Define the metadata for the merged image
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Save the merged image
    merged_output_path = os.path.join(output_directory, f"{data_name}_{aoi_name}.tiff")
    with rasterio.open(merged_output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    print(f"Merged GeoTIFF successfully saved as '{merged_output_path}'")

    # Clean up by closing all tile files and deleting them
    for src in src_files_to_mosaic:
        src.close()
    for tile_path in tile_paths:
        os.remove(tile_path)
    for clipped_tile_path in clipped_tile_paths:
        os.remove(clipped_tile_path)
    print("All individual tile files have been deleted.")

# Calculate the final merged file size
merged_file_size = os.path.getsize(merged_output_path)

# Print total download size and time
end_time = time()
total_runtime = end_time - start_time
print(f"Total merged file size: {merged_file_size / (1024 * 1024):.2f} MB")
print(f"Total runtime: {total_runtime / 60:.2f} minutes")


Tile (0, 0) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_0.tiff'
Tile (0, 1) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_1.tiff'
Tile (0, 2) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_2.tiff'
Tile (0, 3) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_3.tiff'
Tile (0, 4) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_4.tiff'
Tile (0, 5) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_5.tiff'
Tile (1, 0) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_1_0.tiff'
Tile (1, 1) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_1_1.tiff'
Tile (1, 2) successfully downloaded and saved as '/conte

# Download data for US Southeast extent (Run locally)

In [4]:
from IPython.utils import io

with io.capture_output() as captured:
    !pip install rasterio
    !pip install geopandas
    !pip install tqdm
    !pip install requests


In [1]:
import requests
import os
import math
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gpd
from time import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm

# This script downloads raster data for a specified extent as smaller GeoTIFF
# tiles, clips them to the AOI, merges them into a single GeoTIFF file, and
# deletes the individual tiles.

# Data name to download
#data_name = "histosol_percents"
#data_name = "histosol_top20crops"
data_name = "histosol_cultivated"

# Define the URL for the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{data_name}/ImageServer/exportImage"

# Define AOI name:
aoi_name = 'SE'
#aoi_name = 'USA'

# Define AOI GeoJSON file path
aoi_geojson_path = "C:/TerraCarbon/peat/data/SE_peat_aoi.geojson"
#aoi_geojson_path = "C:/TerraCarbon/peat/data/US_peat_aoi.geojson"

# Read AOI GeoJSON to get the geometry
aoi = gpd.read_file(aoi_geojson_path)

# Ensure the AOI is in the same projection as the data (EPSG:3857)
if aoi.crs.to_string() != 'EPSG:3857':
    aoi = aoi.to_crs('EPSG:3857')

# Make the aoi into a bounding box
aoi_bounds = aoi.total_bounds  # [xmin, ymin, xmax, ymax]

# Convert the bounds to a dictionary
bbox = {
    "xmin": aoi_bounds[0],
    "ymin": aoi_bounds[1],
    "xmax": aoi_bounds[2],
    "ymax": aoi_bounds[3]
}

# Pixel size
pixel_size = 30

# Tile size constraints (in pixels)
tile_width = 15000
tile_height = 4100

# Calculate the width and height in meters
tile_width_m = tile_width * pixel_size
tile_height_m = tile_height * pixel_size

# Calculate the number of tiles needed
width = bbox["xmax"] - bbox["xmin"]
height = bbox["ymax"] - bbox["ymin"]
tiles_x = math.ceil(width / tile_width_m)
tiles_y = math.ceil(height / tile_height_m)

# Directory where you want to save the tiles and the merged image
output_directory = "C:/TerraCarbon/peat/output"
tiles_directory = os.path.join(output_directory, "tiles")

# Ensure the directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(tiles_directory, exist_ok=True)

# List to store file paths of downloaded tiles
tile_paths = []

# Setup retry strategy
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)

# Loop over tiles and download each one
total_tiles = tiles_x * tiles_y
start_time = time()

for i in tqdm(range(tiles_x), desc="Downloading tiles", unit="tile"):
    for j in range(tiles_y):
        xmin = bbox["xmin"] + i * tile_width_m
        xmax = min(xmin + tile_width_m, bbox["xmax"])
        ymin = bbox["ymin"] + j * tile_height_m
        ymax = min(ymin + tile_height_m, bbox["ymax"])

        # Calculate the actual tile size in pixels for the request
        actual_tile_width = int((xmax - xmin) / pixel_size)
        actual_tile_height = int((ymax - ymin) / pixel_size)

        # Define the parameters for the exportImage request
        params = {
            "bbox": f"{xmin},{ymin},{xmax},{ymax}",
            "bboxSR": 102100,
            "size": f"{actual_tile_width},{actual_tile_height}",
            "imageSR": 102100,
            "format": "tiff",
            "f": "image"
        }

        # Send the GET request to the server
        try:
            response = http.get(url, params=params, timeout=120)
            response.raise_for_status()  # Raise an exception for HTTP errors

            # Define the output path for the GeoTIFF tile
            tile_path = os.path.join(tiles_directory, f"tile_{i}_{j}.tiff")
            tile_path = os.path.normpath(tile_path)  # Normalize the path to use the correct OS separator

            # Save the GeoTIFF file to the specified directory
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            tile_paths.append(tile_path)
            print(f"Tile ({i}, {j}) successfully downloaded and saved as '{tile_path}'")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download tile ({i}, {j}). Error: {e}")
            print(f"Response content: {response.content if response else 'No response content'}")

# Clip tiles to AOI and save the clipped tiles
clipped_tile_paths = []

for tile_path in tile_paths:
    with rasterio.open(tile_path) as src:
        try:
            out_image, out_transform = mask(src, aoi.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            clipped_tile_path = tile_path.replace(".tiff", f"_{aoi_name}.tiff")
            with rasterio.open(clipped_tile_path, "w", **out_meta) as dest:
                dest.write(out_image)
            clipped_tile_paths.append(clipped_tile_path)
            print(f"Tile {tile_path} successfully clipped and saved as '{clipped_tile_path}'")
        except Exception as e:
            print(f"Failed to clip tile {tile_path}. Error: {e}")

# Merge clipped tiles into a single image
if clipped_tile_paths:
    src_files_to_mosaic = [rasterio.open(fp) for fp in clipped_tile_paths]
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Define the metadata for the merged image
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Save the merged image
    merged_output_path = os.path.join(output_directory, f"{data_name}_{aoi_name}.tiff")
    with rasterio.open(merged_output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    print(f"Merged GeoTIFF successfully saved as '{merged_output_path}'")

    # Clean up by closing all tile files and deleting them
    for src in src_files_to_mosaic:
        src.close()
    for tile_path in tile_paths:
        os.remove(tile_path)
    for clipped_tile_path in clipped_tile_paths:
        os.remove(clipped_tile_path)
    print("All individual tile files have been deleted.")

# Calculate the final merged file size
merged_file_size = os.path.getsize(merged_output_path)

# Print total download size and time
end_time = time()
total_runtime = end_time - start_time
print(f"Total merged file size: {merged_file_size / (1024 * 1024):.2f} MB")
print(f"Total runtime: {total_runtime / 60:.2f} minutes")


Tile (0, 0) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_0.tiff'
Tile (0, 1) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_1.tiff'
Tile (0, 2) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_2.tiff'
Tile (0, 3) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_3.tiff'
Tile (0, 4) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_4.tiff'
Tile (0, 5) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_5.tiff'
Tile (0, 6) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_6.tiff'
Tile (0, 7) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_7.tiff'
Tile (0, 8) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_8.tiff'
Tile (0, 9) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_9.tiff'
Tile (0, 10) success

Tile (0, 16) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_16.tiff'
Tile (1, 0) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_0.tiff'
Tile (1, 1) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_1.tiff'
Tile (1, 2) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_2.tiff'
Tile (1, 3) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_3.tiff'
Tile (1, 4) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_4.tiff'
Tile (1, 5) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_5.tiff'
Tile (1, 6) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_6.tiff'
Tile (1, 7) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_7.tiff'
Tile (1, 8) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_8.tiff'
Tile (1, 9) succes

Tile (1, 16) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_1_16.tiff'
Tile (2, 0) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_0.tiff'
Tile (2, 1) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_1.tiff'
Tile (2, 2) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_2.tiff'
Tile (2, 3) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_3.tiff'
Tile (2, 4) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_4.tiff'
Tile (2, 5) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_5.tiff'
Tile (2, 6) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_6.tiff'
Tile (2, 7) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_7.tiff'
Tile (2, 8) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_8.tiff'
Tile (2, 9) succes

Tile (2, 16) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_2_16.tiff'
Tile (3, 0) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_0.tiff'
Tile (3, 1) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_1.tiff'
Tile (3, 2) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_2.tiff'
Tile (3, 3) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_3.tiff'
Tile (3, 4) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_4.tiff'
Tile (3, 5) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_5.tiff'
Tile (3, 6) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_6.tiff'
Tile (3, 7) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_7.tiff'
Tile (3, 8) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_8.tiff'
Tile (3, 9) succes

Tile (3, 16) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_3_16.tiff'
Tile (4, 0) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_0.tiff'
Tile (4, 1) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_1.tiff'
Tile (4, 2) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_2.tiff'
Tile (4, 3) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_3.tiff'
Tile (4, 4) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_4.tiff'
Tile (4, 5) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_5.tiff'
Tile (4, 6) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_6.tiff'
Tile (4, 7) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_7.tiff'
Tile (4, 8) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_8.tiff'
Tile (4, 9) succes

Tile (4, 16) successfully downloaded and saved as 'C:\TerraCarbon\peat\output\tiles\tile_4_16.tiff'


Tile C:\TerraCarbon\peat\output\tiles\tile_0_0.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_0_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_1.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_1_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_2.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_2_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_3.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_3_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_4.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_4_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_5.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_5_SE.tiff'
Tile C:\TerraCarbon\peat\output\tiles\tile_0_6.tiff successfully clipped and saved as 'C:\TerraCarbon\peat\output\tiles\tile_0_6_SE.tiff'
Tile C:\TerraCarbon\peat\output\ti

# Print class names (Run locally or on COLAB)

In [ ]:
import requests

# Data name to download
name = "histosol_top20crops"
#name = "histosol_percents"


# Define the URL for the Legend operation on the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{name}/ImageServer/legend"

# Define the parameters for the Legend request
params = {
    "f": "json"
}

# Send the GET request to the server
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if 'layers' in data:
        layers = data['layers']
        for layer in layers:
            print(f"Layer Name: {layer['layerName']}")
            for legend in layer['legend']:
                value = legend['values']
                class_name = legend['label']
                print(f"Value: {value}, Class Name: {class_name}")
    else:
        print("No layers found in the response.")
else:
    print(f"Failed to get legend. HTTP status code: {response.status_code}")
    print(f"Response content: {response.content}")


Layer Name: wo_nfs_gtac/histosol_top20crops
Value: ['Corn'], Class Name: Corn
Value: ['Soybeans'], Class Name: Soybeans
Value: ['Spring Wheat'], Class Name: Spring Wheat
Value: ['Winter Wheat'], Class Name: Winter Wheat
Value: ['Oats'], Class Name: Oats
Value: ['Alfalfa'], Class Name: Alfalfa
Value: ['Hay/Non Alfalfa'], Class Name: Hay/Non Alfalfa
Value: ['Other Crops'], Class Name: Other Crops
Value: ['Sugarcane'], Class Name: Sugarcane
Value: ['Onions'], Class Name: Onions
Value: ['Sod/Grass Seed'], Class Name: Sod/Grass Seed
Value: ['Fallow'], Class Name: Fallow
Value: ['Open Water'], Class Name: Open Water
Value: ['Developed'], Class Name: Developed
Value: ['Grassland/Pasture'], Class Name: Grassland/Pasture
Value: ['Herb Wetlands'], Class Name: Herb Wetlands
Value: ['Woody Wetlands'], Class Name: Woody Wetlands
Value: ['Oranges'], Class Name: Oranges
Value: ['Blueberries'], Class Name: Blueberries
Value: ['Cranberries'], Class Name: Cranberries
